In [1]:
import numpy as np
from pycococreatortools import pycococreatortools
import os
import glob
import cv2
from tqdm import tqdm
import json
from PIL import Image


In [2]:
home_path=os.path.expanduser('~')
dir0_with_images_and_ann = os.path.join(home_path, 'Pobrane/images/Weapon-images')
dir1_with_images_and_ann = os.path.join(home_path, 'Pobrane/images/Weapon-images-outdoor')

In [3]:
def get_segmentation(in_dict):
    segmentation = []
    if not len(in_dict['contours']):
        return segmentation
    in_dict['contours'] = sorted(in_dict['contours'], key=len, reverse=True)
    for contour_pack in [in_dict['contours'][0]]:
        for contour in contour_pack:
            segmentation.append(contour['x'])
            segmentation.append(contour['y'])

    
    return [segmentation]

def get_bbox(in_dict):
    left = in_dict['bbox']['left']
    top = in_dict['bbox']['top']
    right = in_dict['bbox']['right']
    bottom = in_dict['bbox']['bottom']
    width = right-left
    height = bottom-top
    return [left, top, width, height]

In [4]:
weapon_id = 1
cats_ids = {"person":2,"weapon":weapon_id, 'People with weapons':2, 'Glock_17':weapon_id ,'45_ACP_Smith_And_Wesson ' :weapon_id, 'mp5k' : weapon_id, 'LeftHandedWeapons':weapon_id}
image_id = 0
ann_id = 0
coco_output = {'images': [], "annotations": [], 'type': "instances"}
coco_output['categories'] = [{"supercategory": "person","id": cats_ids['person'],"name": "person"},
    {"supercategory": "weapon","id": cats_ids['weapon'],"name": "weapon"}]
tags = set()
for src_dir in [dir0_with_images_and_ann, dir1_with_images_and_ann]:
    images_pathes = glob.glob(src_dir+'/*.png')
    for image_path in tqdm(images_pathes):
        image_id += 1
        image = Image.open(image_path)
        parent_dir, image_name  = image_path.split('/')[-2:]
        image_filename_with_parentdir = os.path.join(parent_dir, image_name)
        image_info = pycococreatortools.create_image_info(
        image_id, image_filename_with_parentdir, image.size)
        
        cutted_image_path = '_'.join(image_path.split('_')[:-1])
        annotation_list = glob.glob(cutted_image_path+'_*.json')
        prev_ann_id = ann_id
        for annot_path in annotation_list:
            with open(annot_path, 'r') as f:
                desc = json.load(f)
                if desc['tagName'] not in tags:
                    print(desc['tagName'])
                    tags.add(desc['tagName'])
                
                if desc['tagName'] not in cats_ids:
                    continue
                
                annotation_info = {}
                annotation_info['segmentation'] = get_segmentation(desc)
                if not len(annotation_info['segmentation']):
                    continue
                ann_id += 1
                annotation_info['area'] = desc['area']
                annotation_info['iscrowd'] = 0
                annotation_info['image_id'] = image_id
                annotation_info['bbox'] = get_bbox(desc)
                annotation_info['category_id'] = cats_ids[desc['tagName']]
                annotation_info['id'] =  ann_id
                coco_output['annotations'].append(annotation_info)
        if prev_ann_id != ann_id:
            coco_output["images"].append(image_info)

  0%|          | 1/25454 [00:00<2:42:47,  2.61it/s]

mouse
book
laptop
weapon
People with weapons
table


  0%|          | 2/25454 [00:00<2:40:54,  2.64it/s]

magazine
WineBottle
cell phone


  0%|          | 3/25454 [00:01<2:42:52,  2.60it/s]

...

  0%|          | 30/25454 [00:11<2:42:42,  2.60it/s]

PictureFrame
boots


  0%|          | 31/25454 [00:11<2:40:05,  2.65it/s]

bench


  0%|          | 0/9998 [00:00<?, ?it/s]


In [5]:
annotation_file='/home/m/Pobrane/trainval.json'
os.makedirs(os.path.split(annotation_file)[0], exist_ok=True)
with open(annotation_file, 'w') as f:
    json.dump(coco_output, f)